# Ниязов Алдияр М.19-04 
Задание 1 Предсказание количества комментариев к посту в FaceBook

Произведем выгрузку данных из датасета

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('Dataset/Training/Features_Variant_1.csv', header=None)

Посмотрим на них

In [2]:
data

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,0,1,0
1,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0
2,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,0,0,0,0,0,1,0
3,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,1,0,0,0,0,0,0
4,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,0,1,0,1
40945,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,0,1,0,2
40946,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,1,0,0,72
40947,7170111,70,497000,9,0.0,1881.0,497.200000,269.0,502.318385,0.0,...,0,0,0,0,0,0,0,1,0,28


Так как данные в датасете отсортированы и это негативно скажется на обучении модели, то перемешаем их

In [3]:
data = data.sample(frac=1)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
12145,358398,0,8153,18,0.0,277.0,35.441624,25.0,35.853805,0.0,...,0,0,0,0,0,0,0,0,1,0
10756,441554,0,60660,4,0.0,2328.0,71.828829,42.0,165.413643,0.0,...,0,1,1,0,0,0,0,0,0,2
22793,9108,0,35,8,0.0,40.0,6.416667,4.0,8.365189,0.0,...,0,0,0,0,0,0,0,1,0,3
21103,220749,0,12792,18,0.0,711.0,39.828571,22.0,67.145678,0.0,...,0,1,0,0,1,0,0,0,0,0
22064,888531,1,4469,47,0.0,258.0,93.044118,88.0,41.825419,0.0,...,0,1,0,1,0,0,0,0,0,0


Отделим значения целевого признака в отдельный массив y

In [4]:
y = data[53].values

Оставшиеся признаки сгруппируем в матрицу Х

In [5]:
X = data.loc[:, 0:52].values

Вычислим средние значения признаков (means) и стандартное отклонение (stds) по каждому столбцу матрицы Х. Сгруппируем значения в соответствующие массивы.

In [6]:
means, stds = [np.mean(X, axis=0), np.std(X, axis=0)]

Если у какого-нибудь признака стандартное отклонение равно 0, значит этот признак является константой и не имеет никакого влияния на целевой признак. Определим такой признак в Х и удалим его из выборки:

In [7]:
np.where(stds==0)

(array([37]),)

In [8]:
X = np.delete(X, 37, axis=1)

Пересчитаем means и stds

In [9]:
means, stds = [np.mean(X, axis=0), np.std(X, axis=0)]

Произведем нормировку признаков. От каждого значения матрицы Х отнимем среднее значение по столбцу, а результат поделим на стандартное отклонение этого столбца.

In [10]:
def normalize_features(X):
    for i in range(0, X.shape[0]):
        for j in range(0, X.shape[1]):
            X[i][j] = (X[i][j] - means[j])/stds[j]
    return X

In [11]:
X = normalize_features(X)

Добавим к матрице X столбец из единиц, чтобы не обрабатывать отдельно коэффициент $w_0$ линейной регрессии

In [12]:
X = np.hstack((X, np.ones((len(X), 1))))


Опишем фунциb вычисления метрик качества. Данные показатели характеризует качество применяемой модели

In [13]:
def mserror(y, y_pred):
    return 1/len(y)*sum((y - y_pred) ** 2)

In [14]:
def rmserror(y, y_pred):
    return np.sqrt(mserror(y, y_pred))

In [15]:
def determination(X, y, w):
    return 1 - (sum((y - np.dot(X, w))**2) / sum((y - np.mean(y, axis=0))**2))

Опишем функцию прогноза для линейной модели, принимающую матрицу Х и вектор весов признаков w, возвращающую вектор прогнозов

In [16]:
def linear_prediction(X, w):
    return np.dot(X, w)


Градиент функционала ошибки вычисляется по формуле:

In [17]:
def gradient_step(X, y, w):
    return  np.dot(2/len(y) * X.T, np.add(linear_prediction(X, w), - y))
    

Градиентный спуск будет производиться в цикле по формуле:
    $$\Large w^t = w^{t-1} - lr*gradient step $$ 
    lr - коэффициент шага (learning rate)

In [18]:
def gradient_descent(X, y, w_init, lr, epoch_num):
    weight_dist = np.inf
    w = w_init
    iter_num = 0
    # Склеим матрицу признаков со столбцом из значений целевого признака
    data = np.concatenate((X, y.reshape(len(y), 1)), axis=1)
    # Основной цикл
    for i in range(epoch_num):
        # Перемешаем данные
        np.random.shuffle(data)
        # Разделим таблицу признаков и целевой признак
        X = np.delete(data, 53, axis=1)
        y = np.delete(data, range(53), axis=1).reshape(1, len(y))[0]
        # Пройдемся по датасету batch-ами
        batch_size = 1000
        for batch_idx in range(int(len(y) / batch_size) + 1):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size
            batch = X[start_idx:end_idx]
            y_batch = y[start_idx:end_idx]
            # вычисляем градиент
            grad = gradient_step(batch, y_batch, w)
            # обновляем параметры
            w = w - lr * grad
        
    return w

Для кросс-валидации разделим датасет на 5 частей (примерно по 8190 строк в каждой)

In [19]:
NUM_FOLDS = 5

In [20]:
data_split = np.array_split(X, NUM_FOLDS)

In [21]:
target_split = np.array_split(y, NUM_FOLDS)

In [22]:
def cross_validation(features, target, num_folds):
    metrics = []
    weights = []
    for idx in range(num_folds):
        data_train = []
        target_train = []
        for i in range(num_folds):
            if i != idx:
                data_train.append(data_split[i])
                target_train.append(target_split[i])
            else:
                data_test = data_split[i]
                target_test = target_split[i]
        data_train = np.concatenate(data_train, axis=0)
        target_train = np.concatenate(target_train)
        # тренировка модели
        grad_desc_weights = gradient_descent(data_train, target_train, np.zeros(X.shape[1]), lr=0.01, epoch_num=10)
        # расчет метрик
        RMSE_train = rmserror(target_train, np.dot(data_train, grad_desc_weights))
        RMSE_test = rmserror(target_test, np.dot(data_test, grad_desc_weights))
        R2_train = determination(data_train, target_train, grad_desc_weights)
        R2_test = determination(data_test, target_test, grad_desc_weights)
        # объединение результатов
        metrics.append([RMSE_train, RMSE_test, R2_train, R2_test])
        weights.append(grad_desc_weights)
    
    return metrics, weights

In [23]:
metrics, weights = cross_validation(data_split, target_split, NUM_FOLDS)

Расчитаем средние значения и дисперсию по метрикам:

In [24]:
metrics_mean = []
metrics_std = []

for i in range(4):
    result_list = []
    for metric in metrics:
        result_list.append(metric[i])
    metrics_mean.append(np.mean(result_list))
    metrics_std.append(np.std(result_list))


Объединим результаты по метрикам в таблицу:

In [25]:
data_metrics = {
        'fold 1': metrics[0], 
        'fold 2': metrics[1], 
        'fold 3': metrics[2], 
        'fold 4': metrics[3], 
        'fold 5': metrics[4],
        'mean': metrics_mean,
        'std': metrics_std
       }

df_metrics = pd.DataFrame(data_metrics, index=['RMSE train', 'RMSE test', 'R2 train', 'R2 test'])

Расчитаем средние значения и дисперсию по весам признаков:

In [26]:
weights_mean = []
weights_std = []

for i in range(53):
    result_list = []
    for weight_list in weights:
        result_list.append(weight_list[i])
    weights_mean.append(np.mean(result_list))
    weights_std.append(np.std(result_list))


Объединим результаты по весам признаков в таблицу:

In [27]:
data_weights = {
    'fold 1': weights[0], 
    'fold 2': weights[1], 
    'fold 3': weights[2], 
    'fold 4': weights[3], 
    'fold 5': weights[4],
    'mean': weights_mean,
    'std': weights_std
}
indexes = []
for i in range(len(weights[0])):
    indexes.append(f'w{i}')
    
df_weights = pd.DataFrame(data_weights, index=indexes)

Объединим две таблицы в общую таблицу результатов и сохраним ее в 'result.csv':

In [28]:
result_df = pd.concat([df_metrics, df_weights])
result_df.to_csv('result.csv')

result_df

,fold 1,fold 2,fold 3,fold 4,fold 5,mean,std
RMSE train,29.415323,28.615316,29.925344,30.043308,28.532348,29.306328,0.634782
RMSE test,29.342734,32.832535,26.842417,26.624191,32.259413,29.580258,2.609433
R2 train,0.301459,0.325026,0.324580,0.319124,0.319182,0.317874,0.008589
R2 test,0.359605,0.254517,0.276055,0.288499,0.313564,0.298448,0.036053
w0,0.044334,-1.424275,0.038959,0.078425,0.482111,-0.156089,0.655561
w1,-0.549890,-0.693388,-0.620555,-0.706712,-0.493307,-0.612770,0.081988
w2,-1.709784,-0.662714,-1.089638,-1.235872,-1.856522,-1.310906,0.431635
w3,-0.092567,-0.087067,0.036510,-0.023025,-0.063766,-0.045983,0.047972
w4,-0.050492,-0.361499,-0.053272,-0.373841,-0.463934,-0.260608,0.174057
w5,0.483690,0.367817,0.343436,0.344642,0.561161,0.420149,0.087461


Полученные данные по метрикам качества модели объединим в таблицу Results_mini_batch.xls.
Проанализировав полученные результаты, можно сделать вывод: большая часть результатов соответствуют установленным нормам, что говорит об успешном решении поставленной задачи. 